In [4]:
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('classic')

In [8]:
y = np.array([0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0])
t = np.array([0,0,1,0,0,0,0,0,0,0])

In [9]:
# 오차제곱합 (sum of squares for error. SSE)
def sum_squares_error(y, t) :
    return 0.5 * np.sum((y-t)**2)

In [10]:
sum_squares_error(y,t)

0.09750000000000003

In [11]:
# 교차 엔트로피 오차(cross entoropy error, CEE)
def cross_entropy_error(y,t) :
    # log 에 0이 들어가 값이 무한대가 되는 것을 방지
    delta = 1e-7
    return -np.sum(t * np.log(y + delta))

In [12]:
cross_entropy_error(y,t)

0.510825457099338